# Wrangle and Analyze Data

## Table of Contents
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

In [10]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import config

<a id='gather'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Gather</h2>

In [2]:
df_twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(os.path.join(os.getcwd(),'image-predictions.tsv'), mode='wb') as file:
    file.write(response.content)

df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [19]:
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)

api = tweepy.API(auth)

In [20]:
x = api.get_status('892420643555336193', tweet_mode='extended')

In [39]:
# x._json
pd.io.json.json_normalize(x._json,).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 73 columns):
contributors                               0 non-null object
coordinates                                0 non-null object
created_at                                 1 non-null object
display_text_range                         1 non-null object
entities.hashtags                          1 non-null object
entities.media                             1 non-null object
entities.symbols                           1 non-null object
entities.urls                              1 non-null object
entities.user_mentions                     1 non-null object
extended_entities.media                    1 non-null object
favorite_count                             1 non-null int64
favorited                                  1 non-null bool
full_text                                  1 non-null object
geo                                        0 non-null object
id                                         1 non

<a id='assess'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Assess</h2>

#### Quality
##### `name` table
- abc.

#### Tidiness
- abc.

<a id='clean'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Clean</h2>